<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/mnist/Colab_Notebook_MNIST_Do_Label_Shift_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download predictions from zenodo

In [ ]:
!wget https://zenodo.org/record/3437666/files/test_labels.txt.gz?download=1 -O test_labels.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-0.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-0.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-10.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-10.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-20.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-20.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-30.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-30.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-40.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-40.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-50.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-50.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-60.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-60.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-70.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-70.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-80.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-80.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-90.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-90.txt.gz
!wget https://zenodo.org/record/3437666/files/valid_labels.txt.gz?download=1 -O valid_labels.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-0.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-0.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-10.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-10.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-20.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-20.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-30.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-30.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-40.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-40.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-50.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-50.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-60.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-60.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-70.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-70.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-80.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-80.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-90.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-90.txt.gz

In [ ]:
!gunzip -f *.gz
!md5sum *.txt

In [ ]:

![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

In [ ]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

In [1]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import gzip
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
test_labels = cifarandmnist.read_preds(open("test_labels.txt"))
valid_labels = cifarandmnist.read_preds(open("valid_labels.txt"))

imbalanceadaptername_to_imbalanceadapter = {
    'em': EMImbalanceAdapter(),
    'bbse-hard': BBSEImbalanceAdapter(soft=False),
    'bbse-soft': BBSEImbalanceAdapter(soft=True)}

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4,5,6,7,8,9])),
    #('CVBCTS', CrossValidatedBCTS(
    #             verbose=True,
    #             num_crossvalidation_splits=5,
    #             betas_to_try=[0.0, 1e-7, 1e-6, 1e-5,
    #                           1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]))
])

adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'BCTS'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'BCTS'),
    #('em', 'CVBCTS'),
]

num_trials = 10
seeds = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
dirichlet_alphas_and_samplesize = [
    (0.1,2000), (1.0,2000),
    (0.1,4000), (1.0,4000),
    (0.1,8000), (1.0,8000)]
tweakone_alphas_and_samplesize = [
    (0.01,2000), (0.9,2000),
    (0.01,4000), (0.9,4000),
    (0.01,8000), (0.9,8000)]

print("Dirichlet shift")

(dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 dirichlet_alpha_to_samplesize_to_baselineacc,                        
 metric_to_samplesize_to_calibname_to_unshiftedvals) =\
  cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = dirichlet_alphas_and_samplesize,
    shifttype='dirichlet',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-4000_seed-",
    testglobprefix="testpreacts_model_mnist_set-4000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)

print("Tweak one shift")

(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 tweakone_alpha_to_samplesize_to_baselineacc,                        
 _) = cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = tweakone_alphas_and_samplesize,
    shifttype='tweakone',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-4000_seed-",
    testglobprefix="testpreacts_model_mnist_set-4000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)



Dirichlet shift
Seed 0
Seed 10
Seed 20
Seed 30
Seed 40
Seed 50
Seed 60
Seed 70
Seed 80
Seed 90
Calibration stats
Metric ece
None 0.8572278936569507
TS 0.46577509216140484
BCTS 0.44442307606394915
Metric nll
None 0.09006560327913023
TS 0.08620000635152587
BCTS 0.08528682882235424
Metric jsdiv
None 0.0066070188292113655
TS 0.007404804879082741
BCTS 0.0063929340205031715
On alpha 0.1 sample size 2000
Metric delta_acc
bbse-hard:None 0.019830000000000014 +/- 0.0011549458861782238
bbse-soft:None 0.01991500000000002 +/- 0.001168398173966947
bbse-soft:TS 0.020825000000000024 +/- 0.0012166433165048673
bbse-soft:BCTS 0.02094500000000002 +/- 0.001213307837727471
em:None 0.020725000000000025 +/- 0.0012020473906185778
em:TS 0.02155000000000001 +/- 0.0012461962327438858
em:BCTS 0.021845000000000017 +/- 0.001253070874356921
Metric jsdiv
bbse-hard:None 0.02655341179337332 +/- 0.001458170320476574
bbse-soft:None 0.025288969765058286 +/- 0.0014606843255425014
bbse-soft:TS 0.02143170606429818 +/- 0.00127

In [2]:
import json
import os
file_out = "label_shift_adaptation_results-4000.json"
dict_to_write = {
    "dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "dirichlet_alpha_to_samplesize_to_baselineacc":
     dirichlet_alpha_to_samplesize_to_baselineacc,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals,
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineacc":
     tweakone_alpha_to_samplesize_to_baselineacc
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [3]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results-4000.json.gz").read())
dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
dirichlet_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_baselineacc']
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineacc']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [4]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(seeds)*num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['2000', '4000', '8000'],
        calibnames_in_table=['None', 'TS', 'BCTS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="MNIST Calibration metric differences", label="mnistcalibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    None & 0.09; 1.77 & 0.09; 1.86 & 0.09; 1.9 & 0.007; 0.56 & 0.007; 0.81 & 0.007; 0.89 & 0.857; 1.75 & 0.857; 1.81 & 0.857; 1.8\\
    TS & 0.086; 0.8 & 0.086; 0.95 & 0.086; 1.05 & 0.007; 1.63 & 0.007; 1.84 & 0.007; 1.94 & 0.466; 0.72 & 0.443; 0.64 & 0.42; 0.6\\
    BCTS & \textbf{0.085; 0.43} & \textbf{0.084; 0.19} & \textbf{0.083; 0.05} & 0.006; 0.81 & \textbf{0.005; 0.35} & \textbf{0.004; 0.17} & 0.444; 0.53 & 0.43; 0.55 & 0.42; 0.6\\
  \end{tabular}}
  \caption{MNIST Calibration metric differences}
  \label{tab:mnistcalibrationcomparison}
\end{table*}



In [5]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:BCTS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
           'bbse-soft:TS', 'bbse-soft:BCTS'])])
samplesizes_in_table = ['2000', '4000', '8000']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'BCTS': 'BCTS'}

dirichlet_alphas_in_table = ['0.1', '1.0']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: JS Divergence, dirichlet shift",
    label="mnistjsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="mnistdeltaaccdirichletshift",
    applyunderline=False))

tweakone_alphas_in_table = ['0.01', '0.9']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: JS Divergence, tweakone shift",
    label="mnistjsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="mnistdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    \hline
    EM & None & 0.016; 1.49 & 0.015; 1.59 & 0.014; 1.59 & 0.01; 1.07 & 0.009; 1.18 & 0.008; 1.26\\
    EM & TS & 0.01; 0.76 & 0.009; 0.74 & 0.008; 0.81 & 0.009; 1.05 & 0.009; 1.27 & 0.008; 1.47\\
    EM & BCTS & 0.01; 0.75 & 0.009; 0.67 & 0.008; 0.6 & 0.009; 0.88 & \textbf{0.007; 0.55} & \textbf{0.006; 0.27}\\
    \hline
    \hline
    BBSE-hard & None & 0.027; 2.17 & 0.025; 2.26 & 0.024; 2.32 & 0.011; 1.73 & 0.01; 1.83 & 0.009; 2.03\\
    BBSE-soft & None & 0.025; 1.96 & 0.024; 2.04 & 0.023; 2.13 & 0.01; 1.37 & 0.009; 